In [ ]:
"""
すべてのコーディネートを
train,test, validateの三種に分ける。
そして、画像フォルダとcoordinatesフォルダも分ける
"""

In [2]:
import glob
import random
import shutil
import os
import json
from PIL import Image
import sys
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import can_open_images, open_json
# fix seed
seed_value = 42
random.seed(seed_value)

In [45]:
import importlib
import utils
importlib.reload(utils.util)
from utils.util import can_open_images, open_json

In [10]:
# 画像をすべて移す
dirpath = glob.glob('C:/Users/yuuta/Documents/fashion/IQON3000/IQON3000/**/**/*.jpg')

for fp in dirpath:
    _, img_path = os.path.split(fp)
    img_path = img_path.replace('_m', '')
    shutil.copy(fp, f'C:/Users/yuuta/Documents/fashion/data/images/{img_path}')

In [11]:
# userごとに分ける
dirpath = glob.glob('C:/Users/yuuta/Documents/fashion/IQON3000/IQON3000/**/')

random.shuffle(dirpath)

# 分割の比率を設定
total_length = len(dirpath)
ratio_1 = int(total_length * 0.6)
ratio_2 = int(total_length * 0.8)

# リストを分割
train = dirpath[:ratio_1]
test = dirpath[ratio_1:ratio_2]
validate = dirpath[ratio_2:]

In [49]:

'''
data/{kind}/{user_id}/coordinates.jsonに対して、ファイルを意味のあるコーディネートに絞りコピーしていく関数
'''
def copy_coordinates(filepaths, user_id, kind):
    for fp in filepaths:
        _, coordinate_file_name  = os.path.split(fp)
        # 1. jsonファイルが開けるかの確認
        coordinate = open_json(fp)
        if not coordinate:
            continue

        # 2. そもそもitemsがあるか確認
        if len(coordinate["items"]) == 0:
            continue

        # 3. すべての画像が開けるかの確認
        image_ids = list(map(lambda x: x["itemId"], coordinate["items"]))
        if not can_open_images(image_ids):
            continue
        user_dir_path = f'C:/Users/yuuta/Documents/fashion/data/{kind}/{user_id}'
        if not os.path.exists(user_dir_path):
            os.mkdir(user_dir_path)
        shutil.copy(fp, f'{user_dir_path}/{coordinate_file_name}')

def copy_user_coordinates(dir_path, kind):
    for cnt, user_dirpath in enumerate(dir_path):
        # dir_pathはtailが\\なので二回かける
        if cnt % 3000 == 0:
            print(f'{cnt * 100 / len(dir_path)}')
        _, user_id = os.path.split(os.path.split(user_dirpath)[0])
        user_coordinates_paths = glob.glob(user_dirpath + '**/*_new.json')
        copy_coordinates(user_coordinates_paths, user_id, kind)

def recreate_dataset(kind):
    # ディレクトリが存在する場合は削除
    path = f'C:/Users/yuuta/Documents/fashion/data/{kind}'
    if os.path.exists(path):
        shutil.rmtree(path)

    # 新しいディレクトリを作成
    os.mkdir(path)


In [51]:
copy_user_coordinates(train, 'train')
copy_user_coordinates(test, 'test')
copy_user_coordinates(validate, 'validate')

0.0
0.0
0.0


In [50]:
recreate_dataset('test')
recreate_dataset('train')
recreate_dataset('validate')

In [3]:

# captionの保存
captions = {}
dirpath = glob.glob('C:/Users/yuuta/Documents/fashion/IQON3000/IQON3000/**/')

for dp in dirpath:
    filepaths = glob.glob(dp + '**/*_new.json')
    for fp in filepaths:
        coordinate = open_json(fp)
        if coordinate == None:
            continue
        # print(coordinate)
        try:
            caption_ids = list(map(lambda x: (x["itemId"], x["expressions"]), coordinate["items"]))
        except KeyError as e:
            continue

        for itemId, caption in caption_ids:
            try:
                captions[itemId] = caption[0]
            except IndexError as e:
                continue

with open('C:/Users/yuuta/Documents/fashion/data/captions.json', 'w', encoding='shift-jis') as f:
    json.dump(captions, f, ensure_ascii=False)
